In [1]:
import pandas as pd
import spotipy
import datetime as dt
from random import sample
from secrets import spotify_secrets, localhost_db, postgres_db
from sqlalchemy import create_engine
from spotipy.oauth2 import SpotifyOAuth
from queries import *

df_a = pd.read_csv('data/TopArtists.csv')
df_t = pd.read_csv('data/TopTracks.csv')
df_g = pd.read_csv('data/TopGenres.csv')
df_m = pd.read_csv('data/MusicFeatures.csv')

In [2]:
u2_a = df_a.loc[df_a['user_id'] == 12120382831]
u2_t = df_t.loc[df_t['user_id'] == 12120382831]
u2_g = df_g.loc[df_g['user_id'] == 12120382831]
u2_m = df_m.loc[df_m['user_id'] == 12120382831]

In [3]:
u2_a.head()

,user_id,rank,artist_id,timeframe,artist,genres,artist_url,artist_image,popularity
1,12120382831,1,5K4W6rqBFWDnAN6FQUkS6x,Long,Kanye West,chicago rap; rap,https://open.spotify.com/artist/5K4W6rqBFWDnAN...,https://i.scdn.co/image/bd1c6fdf3705cf9b7d0c8a...,90
2,12120382831,2,2elBjNSdBE2Y3f0j1mjrql,Long,Jay Chou,c-pop; mandopop; taiwan pop; zhongguo feng,https://open.spotify.com/artist/2elBjNSdBE2Y3f...,https://i.scdn.co/image/0e5952203ea0140aa5687f...,73
4,12120382831,3,73sIBHcqh3Z3NyqHKZ7FOL,Long,Childish Gambino,atl hip hop; hip hop; pop rap; rap,https://open.spotify.com/artist/73sIBHcqh3Z3Ny...,https://i.scdn.co/image/2e1d974c14fb66eb5050bf...,80
7,12120382831,4,1Xyo4u8uXC1ZmMpatF05PJ,Long,The Weeknd,canadian contemporary r&b; canadian pop; pop,https://open.spotify.com/artist/1Xyo4u8uXC1ZmM...,https://i.scdn.co/image/d9a875c37277c35b94c60c...,94
8,12120382831,5,50co4Is1HCEo8bhOyUWKpn,Long,Young Thug,atl hip hop; atl trap; gangster rap; melodic r...,https://open.spotify.com/artist/50co4Is1HCEo8b...,https://i.scdn.co/image/69ef089edd03d6d5455ed6...,90


In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_secrets["Client Id"],
                                               client_secret=spotify_secrets["Client Secret"],
                                               redirect_uri="http://localhost:8892/callback",
                                               scope="user-top-read"))

In [5]:
def recommend_artists(df):
    # sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
    user_id = df['user_id'].unique()[0]
    recom_list = []
    seed_artists = df['artist_id'].iloc[:5].tolist()
    recommend_time = dt.datetime.now()
    for seed in seed_artists:
        recom = sp.artist_related_artists(seed)
        for a in recom['artists']:
            if a['id'] not in df['artist_id'].iloc[:20]:
                this_recom = {
                    'user_id': user_id,
                    'artist_id': a['id'],
                    'artist': a['name'],
                    'genres': "; ".join(g for g in a['genres']),
                    'artist_url': a['external_urls']['spotify'],
                    'artist_image': a['images'][0]['url'],
                    'popularity': a['popularity'],
                    'date_recommended': recommend_time
                }
                recom_list.append(this_recom)
    if len(recom_list) >= 20:
        return pd.DataFrame.from_dict(recom_list).sample(n=20)
    else:
        return pd.DataFrame.from_dict(recom_list)

def recommend_tracks(df):
    # sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
    user_id = df['user_id'].unique()[0]
    seed_tracks = df['track_id'].iloc[:5].tolist()
    recom = sp.recommendations(seed_tracks=seed_tracks, limit=50)
    recom_list = []
    recommend_time = dt.datetime.now()
    for t in recom['tracks']:
        if t['id'] not in df['track_id'].iloc[:20]:
            this_recom = {
                'user_id': user_id,
                'track_id': t['id'],
                'track_id': t['id'],
                'track': t['name'],
                'artists': "; ".join(a['name'] for a in t['artists']),
                'album': t['album']['name'],
                'album_image': t['album']['images'][0]['url'],
                'release_date': t['album']['release_date'],
                'track_url': t['external_urls']['spotify'],
                'date_recommended': recommend_time
            }
            recom_list.append(this_recom)
        if len(recom_list) >= 30:
            break
    return pd.DataFrame.from_dict(recom_list)

In [6]:
df_ra = recommend_artists(u2_a.loc[u2_a['timeframe'] == 'Short'])
df_ra.head()

,user_id,artist_id,artist,genres,artist_url,artist_image,popularity,date_recommended
99,12120382831,5upXI71QNPjLwir1YZgKEx,cinema staff,j-poprock; j-rock,https://open.spotify.com/artist/5upXI71QNPjLwi...,https://i.scdn.co/image/3278c5f04a58a5667f7f87...,51,2021-01-15 18:05:17.132797
48,12120382831,13ubrt8QOOCPljQ2FL1Kca,A$AP Rocky,east coast hip hop; hip hop; rap; trap; underg...,https://open.spotify.com/artist/13ubrt8QOOCPlj...,https://i.scdn.co/image/cce607c4d975ae49d68b48...,85,2021-01-15 18:05:17.132797
69,12120382831,4sTQVOfp9vEMCemLw50sbu,Galantis,big room; dance pop; edm; electro house; pop; ...,https://open.spotify.com/artist/4sTQVOfp9vEMCe...,https://i.scdn.co/image/2df5fb887384d10f117938...,76,2021-01-15 18:05:17.132797
52,12120382831,0Y4inQK6OespitzD6ijMwb,Freddie Gibbs,alternative hip hop; hip hop; indiana hip hop;...,https://open.spotify.com/artist/0Y4inQK6Oespit...,https://i.scdn.co/image/b0e5695a33f4b5fe649020...,73,2021-01-15 18:05:17.132797
10,12120382831,6beUvFUlKliUYJdLOXNj9C,King Princess,dance pop; electropop; indie pop; indie poptim...,https://open.spotify.com/artist/6beUvFUlKliUYJ...,https://i.scdn.co/image/e1636b720d17e3ac655421...,70,2021-01-15 18:05:17.132797


In [9]:
'4sTQVOfp9vEMCemLw50sbu' in df_ra['artist_id'].values

True

In [10]:
df_rt = recommend_tracks(u2_t.loc[u2_t['timeframe'] == 'Short'])
df_rt.head()

,user_id,track_id,track,artists,album,album_image,release_date,track_url,date_recommended
0,12120382831,1eKvw144NYPlDjR0wQoGLW,Where's My Love - Acoustic,SYML,Where's My Love,https://i.scdn.co/image/ab67616d0000b27399a8a8...,2017-07-07,https://open.spotify.com/track/1eKvw144NYPlDjR...,2021-01-13 23:45:38.295668
1,12120382831,5v8aEcHcaJIGYpfVVPtJjI,Wires,The Neighbourhood,I'm Sorry...,https://i.scdn.co/image/ab67616d0000b273ee14a6...,2013-05-08,https://open.spotify.com/track/5v8aEcHcaJIGYpf...,2021-01-13 23:45:38.295668
2,12120382831,3pXF1nA74528Edde4of9CC,Don't,Bryson Tiller,T R A P S O U L,https://i.scdn.co/image/ab67616d0000b273d5f3ce...,2015-10-02,https://open.spotify.com/track/3pXF1nA74528Edd...,2021-01-13 23:45:38.295668
3,12120382831,0baNzeUcPQnQSagpe8T0mD,worldstar money (interlude),Joji,In Tongues (Deluxe),https://i.scdn.co/image/ab67616d0000b2738e947f...,2018-02-14,https://open.spotify.com/track/0baNzeUcPQnQSag...,2021-01-13 23:45:38.295668
4,12120382831,2qpacEyFxmbxCpIEqZkqvC,You Are the Right One,Sports,Naked All the Time,https://i.scdn.co/image/ab67616d0000b27327e52d...,2015-07-24,https://open.spotify.com/track/2qpacEyFxmbxCpI...,2021-01-13 23:45:38.295668


In [11]:
df_ra['user_id'] = df_ra['user_id'].astype(str)
df_rt['user_id'] = df_rt['user_id'].astype(str)

In [14]:
engine = create_engine(localhost_db)
df_ra[['user_id', 'artist_id', 'date_recommended']].to_sql('RecommendArtists', engine, index=False, if_exists='replace')
df_rt[['user_id', 'track_id', 'date_recommended']].to_sql('RecommendTracks', engine, index=False, if_exists='replace')
df_ra.drop(columns=['user_id', 'date_recommended']).to_sql('TempArtists', engine, index=False, if_exists='replace')
df_rt.drop(columns=['user_id', 'date_recommended']).to_sql('TempTracks', engine, index=False, if_exists='replace')
engine.dispose()

In [15]:
engine = create_engine(postgres_db)
df_ra[['user_id', 'artist_id', 'date_recommended']].to_sql('RecommendArtists', engine, index=False, if_exists='replace')
df_rt[['user_id', 'track_id', 'date_recommended']].to_sql('RecommendTracks', engine, index=False, if_exists='replace')
engine.dispose()

In [16]:
engine = create_engine(localhost_db)
df = pd.read_sql_query('select * from "Users"', engine)
engine.dispose()
df.head()

,user_id,display_name,spotify_url,image_url,followers,date_created,last_updated,last_recommended
0,12120382831,Bin Xuan Kong,https://open.spotify.com/user/12120382831,https://scontent-hkt1-1.xx.fbcdn.net/v/t1.0-1/...,53,2021-01-05 16:01:39.106837,2021-01-05 16:01:39.106848,None
1,12153521253,Thivya Dharishinie,https://open.spotify.com/user/12153521253,https://scontent-amt2-1.xx.fbcdn.net/v/t1.0-1/...,46,2021-01-05 09:12:10.904861,2021-01-05 09:23:00.167209,None
2,1279967390,Jae Sheng Ang,https://open.spotify.com/user/1279967390,https://scontent-ams4-1.xx.fbcdn.net/v/t1.0-1/...,122,2021-01-06 15:50:28.900338,2021-01-06 15:50:28.900356,None
3,12179805550,Clement Tan,https://open.spotify.com/user/12179805550,https://scontent-cdt1-1.xx.fbcdn.net/v/t1.0-1/...,61,2021-01-06 15:51:34.460870,2021-01-06 15:53:41.620661,None


In [222]:
import re
import random
import lyricsgenius

TRY_COUNT = 5

def get_new_of_day():
    a = artist_of_day()
    s = song_of_day()
    l = lyric_of_day()
    return a, s, l

def artist_of_day():
    engine = create_engine(localhost_db)
    for i in range(TRY_COUNT):
        df = pd.read_sql_query('SELECT * FROM "Artists" TABLESAMPLE BERNOULLI(1) LIMIT 1;', engine)
        if len(df) > 0:
            engine.dispose()
            return df
    enginge.dispose()
    return None

def song_of_day():
    engine = create_engine(localhost_db)
    for i in range(TRY_COUNT):
        df = pd.read_sql_query('SELECT * FROM "Tracks" TABLESAMPLE BERNOULLI(1) LIMIT 1;', engine)
        if len(df) > 0:
            engine.dispose()
            return df
    engine.dispose()
    return None

def lyric_of_day():
    engine = create_engine(localhost_db)
    for i in range(TRY_COUNT):
        df = pd.read_sql_query('SELECT * FROM "Tracks" TABLESAMPLE BERNOULLI(1) LIMIT 1;', engine)
        if len(df) > 0:
            for _, row in df.iterrows():
                if re.search('[a-zA-Z0-9]', row['track']) is not None:
                    lyrics = get_lyrics(row['track'], row['artists'].split(';')[0])
                    if isinstance(lyrics, str):
                        engine.dispose()
                        row['lyrics'] = lyrics
                        return row
    engine.dispose()
    return None

def get_lyrics(track, artist):
    genius = lyricsgenius.Genius(token)
    try:
        song = genius.search_song(track, artist)
        lyrics = '['.join(song.lyrics.split(']')).split('[')
        verse = random.choice([l for l in lyrics if len(l) > 100])
        return verse
    except:
        return None

In [195]:
a = artist_of_day()
t = song_of_day()
l = lyric_of_day()

Searching for "Modern Loneliness - Vaundy Remix" by Lauv...
Done.


In [232]:
df_temp = pd.DataFrame([[a['artist_id'][0], t['track_id'][0], l['track_id'], dt.datetime.now()]], \
                       columns=['artist_id', 'track_id', 'lyrics_id', 'date_created'])
df_temp

,artist_id,track_id,lyrics_id,date_created
0,3WGpXCj9YhhfX11TToZcXP,1S9Kc1FlQoK3xh53mP5Udt,1dFkD1JfRMzwO6hwUsE8aS,2021-01-16 00:44:30.017158


In [239]:
engine = create_engine(postgres_db)
# df_temp.to_sql('DailyMix', engine, index=False)
engine.execute('UPDATE "Tracks" SET lyrics = %(lyrics)s WHERE track_id = %(track_id)s', lyrics=l['lyrics'], \
        track_id=l['track_id'])
engine.dispose()

In [217]:
def get_of_the_day():
    engine = create_engine(localhost_db)
    df = pd.read_sql_query('SELECT * FROM "DailyMix" ORDER BY date_created DESC LIMIT 1', engine)
    if (dt.datetime.now() - df['date_created'].item()).days >= 1:
        artist, track, lyrics = get_new_of_day()
    else:
        artist = pd.read_sql_query('SELECT * FROM "Artists" WHERE artist_id = %(i)s', engine, params={'i': df['artist_id'].item()})
        track = pd.read_sql_query('SELECT * FROM "Tracks" WHERE track_id = %(i)s', engine, params={'i': df['track_id'].item()})
        lyrics = pd.read_sql_query('SELECT * FROM "Tracks" WHERE track_id = %(i)s', engine, params={'i': df['lyrics_id'].item()})
    return artist, track, lyrics

In [223]:
a, t, l = get_of_the_day()

Searching for "Spectrum" by Zedd...
Done.


In [224]:
a

,artist_id,artist,genres,artist_url,artist_image,popularity
0,3WGpXCj9YhhfX11TToZcXP,Troye Sivan,australian pop; dance pop; electropop; pop; po...,https://open.spotify.com/artist/3WGpXCj9YhhfX1...,https://i.scdn.co/image/457ae8b1846ae6f55ec560...,80


In [225]:
t

,track_id,track,artists,album,album_image,release_date,track_url,lyrics
0,1S9Kc1FlQoK3xh53mP5Udt,Pursue / All I Need Is You,Hillsong Worship; Hillsong Young & Free,OPEN HEAVEN / River Wild (Deluxe),https://i.scdn.co/image/ab67616d0000b2730f6f44...,2015-10-16,https://open.spotify.com/track/1S9Kc1FlQoK3xh5...,None


In [226]:
l

track_id                                   1dFkD1JfRMzwO6hwUsE8aS
track                                                    Spectrum
artists                                        Zedd; Matthew Koma
album                                                     Clarity
album_image     https://i.scdn.co/image/ab67616d0000b273941dd3...
release_date                                           2012-01-01
track_url       https://open.spotify.com/track/1dFkD1JfRMzwO6h...
lyrics          \nBreathing you in when I want you out\nFindin...
Name: 0, dtype: object